In [1]:
import pandas as pd
import numpy as np
import re

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [62]:
raw_data_path = '../data/raw/'

name = '17punto10'
reviews_raw = pd.read_csv(raw_data_path + 'collected_reviews_' + name + '.csv')
resumme_raw = pd.read_csv(raw_data_path + 'resumme_' + name + '.csv')
display(resumme_raw)
display(reviews_raw.sample(5))

,stars,reviews
0,5,571
1,4,191
2,3,35
3,2,13
4,1,13


,author,local_guide_info,rating,review,date,text_backup
21,Jonathan LG,Local Guide · 34 reseñas · 96 fotos,5 estrellas,Fuimos el fin de semana a Zaragoza y nos gusta...,Hace un año,Jonathan LG\nLocal Guide · 34 reseñas · 96 fot...
11,Gustavo Novellon,Local Guide · 69 reseñas · 985 fotos,5 estrellas,Volvemos al 17punto10 y… que gozada!!!\n\nUn s...,Hace un año,Gustavo Novellon\nLocal Guide · 69 reseñas · 9...
25,isabel g. b.,Local Guide · 21 reseñas · 2 fotos,5 estrellas,El pasado sábado descubrimos este estupendo re...,Hace 3 meses,isabel g. b.\nLocal Guide · 21 reseñas · 2 fot...
16,Celia Lahuerta,Local Guide · 104 reseñas · 269 fotos,5 estrellas,Un imprescindible en Zaragoza!\nExcelente cali...,Hace un año,Celia Lahuerta\nLocal Guide · 104 reseñas · 26...
18,Gema Pérez,Local Guide · 54 reseñas · 192 fotos,5 estrellas,Atención y comida maravillosa.\nNo sabría eleg...,Hace un año,Gema Pérez\nLocal Guide · 54 reseñas · 192 fot...


In [63]:
raw_data_path = '../data/raw/'
reviews_raw = pd.read_csv(raw_data_path + 'collected_data.csv')
reviews_raw.drop(columns=['price', 'food', 'service', 'atmosphere', 'recommendations'], inplace=True)
display(reviews_raw.sample(5))

,author,local_guide_info,rating,review,date,text_backup
324,Ilie Alex,Local Guide · 8 reseñas · 17 fotos,5 estrellas,NaN,Hace 4 meses,Ilie Alex\nLocal Guide · 8 reseñas · 17 fotos\...
13,Natalia C.F.,Local Guide · 263 reseñas · 509 fotos,3 estrellas,No esta mal. Por el precio ponen una tapa gran...,Hace un mes,Natalia C.F.\nLocal Guide · 263 reseñas · 509 ...
303,Miguel Angel Ossorio Rodríguez,Local Guide · 19 reseñas · 9 fotos,4 estrellas,NaN,Hace un mes,Miguel Angel Ossorio Rodríguez\nLocal Guide · ...
236,Izaro Iraolagoitia,1 reseña · 2 fotos,5 estrellas,NaN,Hace 4 años,Izaro Iraolagoitia\n1 reseña · 2 fotos\n\n\n...
41,--,Local Guide · 189 reseñas · 4 fotos,5 estrellas,"A pesar de estar petado, el trato de los camar...",Hace 10 meses,--\nLocal Guide · 189 reseñas · 4 fotos\n\n\...


In [64]:
restaurant_search_words = {
    'service': r'Servicio\n([^\n]+)',
    'meal_type': r'Tipo de comida\n([^\n]+)',
    'price_per_person': r'Precio por persona\n([0-9€\- ]+)',
    'food_score': r'Comida: (\d+)',
    'service_score': r'Servicio: (\d+)',
    'atmosphere_score': r'Ambiente: (\d+)',
    'recommended': r'Platos recomendados\n([^\n]+)'
}

In [71]:
def extractRestaurantDetailsFromReview(sample, search_words=None, verbose=False):
    clean_text = re.sub(r'\\ue[0-9a-f]{3}', '', sample)
    clean_text = re.sub(r'\n+', '\n', clean_text)
    clean_text = clean_text.strip()

    # Store extracted values
    extracted_values = []

    # Loop through search words to extract values dynamically
    for key, regex in search_words.items():
        match = re.search(regex, clean_text)
        value = match.group(1) if match else ''
        extracted_values.append(value)

    return extracted_values

def applyExtractDetails(df, search_words=None):
    column_names = list(search_words.keys())
    df[column_names] = df['text_backup'].apply(lambda x: pd.Series(extractRestaurantDetailsFromReview(x, search_words=search_words)))
    return df

def extractReviewCount(text):
    if isinstance(text, str):  # Verify if its a string
        match = re.search(r'(\d+)\s+reseñas', text)
        if match:
            return int(match.group(1))
    return None

def extractStarRating(text):
    match = re.search(r'(\d+)\s+estrellas', text)
    if match:
        return int(match.group(1))
    return None

In [73]:
reviews = reviews_raw.copy()
reviews['local_guide_reviews'] = reviews['local_guide_info'].apply(extractReviewCount)
reviews['rating_score'] = reviews['rating'].apply(extractStarRating)

reviews = applyExtractDetails(reviews, search_words = restaurant_search_words)
reviews.drop(columns = ['text_backup', 'local_guide_info'], inplace = True)
display(reviews.head(30))


,author,rating,review,date,local_guide_reviews,rating_score,service,meal_type,price_per_person,food_score,service_score,atmosphere_score,recommended
0,LO TIZ,5 estrellas,Fantástico ambiente. Camareros rápidos y amabl...,Hace una semana,99.0,5.0,,,1-10 €,5,5,5,"Tapas Y Dobles de Cerveza, Patatas Bravas O Al..."
1,Blvck Kinder,5 estrellas,El Bar Sierra en Madrid es un verdadero tesoro...,Hace 2 meses,35.0,5.0,,,1-10 €,5,5,5,Tapas Y Dobles de Cerveza
2,Arkaitz fernandez sanabria,5 estrellas,Tapas absolutamente de locos. Por apenas 3.30 ...,Hace 5 meses,198.0,5.0,,,,5,5,5,"Tapas Y Dobles de Cerveza, Patatas Bravas O Al..."
3,ANGELA “LocuraTodoLoCura”,5 estrellas,La leche!! Este sitio es el mejor con diferenc...,Hace una semana,84.0,5.0,,,,5,5,5,Tapas Y Dobles de Cerveza
4,Claudia M. Simal.,5 estrellas,"Por el precio nos gustó mucho, los tamaños de ...",Hace 6 meses,308.0,5.0,,,,5,3,3,
5,David,5 estrellas,En la zona de Chamberí en Madrid. La estación ...,Hace 2 meses,278.0,5.0,,,1-10 €,5,4,4,
6,César,4 estrellas,Os cuento según mi opinión:\n\nContras:\nSitio...,Hace 3 meses,61.0,4.0,,,1-10 €,4,3,4,
7,Dxvid Martos,5 estrellas,"El camarero mu majo, el baño limpio y las tapa...",Hace un mes,69.0,5.0,,,1-10 €,5,5,5,Patatas Bravas O Alioli y Tapa de Macarrones
8,jose maria jarillo,5 estrellas,"Gran bar de tapas en la zona baja de chamberi,...",Hace 4 meses,159.0,5.0,,,10-20 €,5,5,5,"Tapas Y Dobles de Cerveza, Patatas Bravas O Al..."
9,David Cabeza Domínguez,5 estrellas,Hemos comido totalmente con dos consumiciones ...,Hace 3 meses,323.0,5.0,,,1-10 €,4,5,5,Tapas Y Dobles de Cerveza


In [ ]:
# Fill NA values

reviews['local_guide_reviews'].fillna(1, inplace = True)
reviews['rating_score'].fillna(1, inplace = True)
